<a href="https://colab.research.google.com/github/JBenjaminGM/ProyectoFinal-Analisis-de-Sentimiento/blob/main/Limpieza_de_Comentarios_con_PySpark_y_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Instalación de Pyspark, PyDrive y Java 8.0*

In [110]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u382-ga-1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


*Importa las librerias Pyspark y se crea la sesión de conexión a PySpark*

In [111]:
import pyspark
# Carga ufnciones extra
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('primeros_pasos').getOrCreate()

*Carga de datos en crudo de los comentarios Twitter desde un archivo csv*

In [117]:
df_tweets = spark.read.csv('CSV_primitivo.csv', header = True, encoding='latin1',sep = ';', inferSchema=True)

In [118]:
df_tweets.show()

+----+----------------+--------+---------+---------------+--------------------+----------------+
|Item|        Username|Comments|Favorites|           Date|               Tweet|Author Followers|
+----+----------------+--------+---------+---------------+--------------------+----------------+
|   1|  Aventurero8415|    NULL|     NULL|6/10/2023 19:27|@larepublica_pe @...|             369|
|   2|       Azathot00|    NULL|     NULL|6/10/2023 15:09|@larepublica_pe @...|              24|
|   3|       A87385106|    NULL|     NULL|8/10/2023 00:53|@larepublica_pe @...|               1|
|   4| AdrianCarmelo01|    NULL|        1|6/10/2023 20:44|@larepublica_pe @...|               6|
|   5|   AlbertoPeUsEs|    NULL|     NULL|7/10/2023 01:21|@larepublica_pe @...|               2|
|   6| Alejand11416630|    NULL|        1|6/10/2023 17:46|@larepublica_pe @...|               1|
|   7| Alejand45315784|    NULL|     NULL|8/10/2023 00:43|@larepublica_pe @...|              85|
|   8|  tabernero_alex|    NUL

In [119]:
df_tweets.count()

140

*Selección de columnas para analisis de datos*

In [121]:
df_tweets = df_tweets.select('Username', 'Comments', 'Favorites', 'Date', 'Tweet','Author Followers')
df_tweets.show(140)

+----------------+--------+---------+---------------+--------------------+----------------+
|        Username|Comments|Favorites|           Date|               Tweet|Author Followers|
+----------------+--------+---------+---------------+--------------------+----------------+
|  Aventurero8415|    NULL|     NULL|6/10/2023 19:27|@larepublica_pe @...|             369|
|       Azathot00|    NULL|     NULL|6/10/2023 15:09|@larepublica_pe @...|              24|
|       A87385106|    NULL|     NULL|8/10/2023 00:53|@larepublica_pe @...|               1|
| AdrianCarmelo01|    NULL|        1|6/10/2023 20:44|@larepublica_pe @...|               6|
|   AlbertoPeUsEs|    NULL|     NULL|7/10/2023 01:21|@larepublica_pe @...|               2|
| Alejand11416630|    NULL|        1|6/10/2023 17:46|@larepublica_pe @...|               1|
| Alejand45315784|    NULL|     NULL|8/10/2023 00:43|@larepublica_pe @...|              85|
|  tabernero_alex|    NULL|     NULL|6/10/2023 15:27|@larepublica_pe @...|      

*Mostramos las forma de los datos según inferencia de pySpark *

In [122]:
df_tweets.printSchema()

root
 |-- Username: string (nullable = true)
 |-- Comments: integer (nullable = true)
 |-- Favorites: integer (nullable = true)
 |-- Date: string (nullable = true)
 |-- Tweet: string (nullable = true)
 |-- Author Followers: integer (nullable = true)



In [123]:
df_tweets.count()

140

In [124]:
#Limpieza de datos duplicados..
#df_tweets.count()
df_tweets=df_tweets.dropDuplicates()

In [125]:
df_tweets.count()

140

In [126]:
# Limpieza de datos nulos - Elimina una fila siempre que la columna "Tweet text contega registro nulos.".
df_tweets.na.drop(subset=["Tweet"]).show(truncate=False)

df_tweets.count()

+---------------+--------+---------+---------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------+
|Username       |Comments|Favorites|Date           |Tweet                                                                                                                                                                                                                                                                                                              |Author Followers|
+---------------+--------+---------+---------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

140

In [127]:
df_tweets.select('Tweet').show()
df_tweets.count()

+--------------------+
|               Tweet|
+--------------------+
|"@larepublica_pe ...|
|?? Policías podrá...|
|@larepublica_pe @...|
|@larepublica_pe @...|
|@larepublica_pe @...|
|@larepublica_pe @...|
|@larepublica_pe @...|
|@larepublica_pe @...|
|"@larepublica_pe ...|
|@larepublica_pe @...|
|@larepublica_pe @...|
|@larepublica_pe @...|
|@larepublica_pe @...|
|@larepublica_pe @...|
|@larepublica_pe @...|
|@larepublica_pe @...|
|@larepublica_pe @...|
|@larepublica_pe @...|
|@larepublica_pe @...|
|@larepublica_pe @...|
+--------------------+
only showing top 20 rows



140

In [128]:
#Completa registros nulls
df_tweets2=df_tweets.fillna({'Username': 'Desconocido','Date':0,'Comments':0,'Favorites':0,'Author Followers':0})

In [129]:
#Reemplazar una cadena "@larepublica_pe @jesicaleon" por otra cadena.

from pyspark.sql.functions import regexp_replace
df_tweets2=df_tweets2.withColumn('Tweet', regexp_replace('Tweet', '@larepublica_pe @jesicaleon', ''))
df_tweets2=df_tweets2.withColumn('Tweet', regexp_replace('Tweet', '@larepublica_pe', ''))
df_tweets2=df_tweets2.withColumn('Tweet', regexp_replace('Tweet', '@jesicaleon', ''))

In [130]:
df_tweets2.show()

+---------------+--------+---------+---------------+--------------------+----------------+
|       Username|Comments|Favorites|           Date|               Tweet|Author Followers|
+---------------+--------+---------+---------------+--------------------+----------------+
|     turdepeish|       3|        8|6/10/2023 12:24|" Hace rato era, ...|             140|
| larepublica_pe|     170|      395|6/10/2023 12:05|?? Policías podrá...|         3013243|
|     lazaro1975|      13|        7|6/10/2023 14:33| Desde hace casi ...|             262|
| cazarte_carlos|       1|        5|6/10/2023 13:55| ,  maravilla, la...|             289|
|   gabrielal4m0|       2|       33|6/10/2023 13:20| @jfowks  La modi...|            1059|
|    YorshCarlin|       1|        8|6/10/2023 14:27| Una puerta abier...|             106|
|         rommyb|       3|       51|6/10/2023 12:29| ¿Detendrán a del...|             658|
|    Junior_0311|       1|       20|6/10/2023 13:54| Ptm o sea a quie...|            2510|

In [131]:
df_tweets2.count()

140

In [132]:
#Reemplazar una cadena "@larepublica_pe @jesicaleon" por otra cadena.

import pandas as pd
import numpy as np


# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

result_comentarios= df_tweets2.select("*").toPandas()

print(result_comentarios)

            Username  Comments  Favorites             Date  \
0         turdepeish         3          8  6/10/2023 12:24   
1     larepublica_pe       170        395  6/10/2023 12:05   
2         lazaro1975        13          7  6/10/2023 14:33   
3     cazarte_carlos         1          5  6/10/2023 13:55   
4       gabrielal4m0         2         33  6/10/2023 13:20   
..               ...       ...        ...              ...   
135        z_bird986         0          0           6 oct.   
136  @cazarte_carlos         0          0           6 oct.   
137  MiguelA04899081         0          0  6/10/2023 14:42   
138  LuisAri77761564         0          0  6/10/2023 21:50   
139       Morga_Ezra         1          0  6/10/2023 16:51   

                                                 Tweet  Author Followers  
0    " Hace rato era, no faltaran los sjw a decir "...               140  
1    ?? Policías podrán retener a civiles hasta por...           3013243  
2     Desde hace casi 30 años 

In [138]:
result_comentarios = result_comentarios.drop(1)
result_comentarios

,Username,Comments,Favorites,Date,Tweet,Author Followers
0,turdepeish,3,8,6/10/2023 12:24,hace rato era no faltaran los sjw a decir ayy...,140
2,lazaro1975,13,7,6/10/2023 14:33,desde hace casi 30 años salgo siempre con mis...,262
3,cazarte_carlos,1,5,6/10/2023 13:55,maravilla la penepe con competencias de jue...,289
4,gabrielal4m0,2,33,6/10/2023 13:20,jfowks la modificación del código no ha afec...,1059
5,YorshCarlin,1,8,6/10/2023 14:27,una puerta abierta al abuso policialincluso t...,106
...,...,...,...,...,...,...
135,z_bird986,0,0,6 oct.,a quienes vean sospechosos o a quien creen qu...,0
136,@cazarte_carlos,0,0,6 oct.,la modificación del código no ha afectado esa...,0
137,MiguelA04899081,0,0,6/10/2023 14:42,¿acusar y denunciar cuando están coimeando te...,0
138,LuisAri77761564,0,0,6/10/2023 21:50,todo es negociable no,1


In [139]:
from pandas import DataFrame
import string
result_comentarios[['Tweet']] = result_comentarios[['Tweet']].astype(str).apply(lambda x: x.str.lower())#paso los datos a minuscula
for puntuacion in string.punctuation:
    result_comentarios['Tweet'] = result_comentarios['Tweet'].str.replace(puntuacion, '')

<ipython-input-139-b2d52a9432fa>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  result_comentarios['Tweet'] = result_comentarios['Tweet'].str.replace(puntuacion, '')


In [140]:
result_comentarios

,Username,Comments,Favorites,Date,Tweet,Author Followers
0,turdepeish,3,8,6/10/2023 12:24,hace rato era no faltaran los sjw a decir ayy...,140
2,lazaro1975,13,7,6/10/2023 14:33,desde hace casi 30 años salgo siempre con mis...,262
3,cazarte_carlos,1,5,6/10/2023 13:55,maravilla la penepe con competencias de jue...,289
4,gabrielal4m0,2,33,6/10/2023 13:20,jfowks la modificación del código no ha afec...,1059
5,YorshCarlin,1,8,6/10/2023 14:27,una puerta abierta al abuso policialincluso t...,106
...,...,...,...,...,...,...
135,z_bird986,0,0,6 oct.,a quienes vean sospechosos o a quien creen qu...,0
136,@cazarte_carlos,0,0,6 oct.,la modificación del código no ha afectado esa...,0
137,MiguelA04899081,0,0,6/10/2023 14:42,¿acusar y denunciar cuando están coimeando te...,0
138,LuisAri77761564,0,0,6/10/2023 21:50,todo es negociable no,1


In [141]:
result_comentarios.isnull().sum().sum()

0

In [142]:
#Descargar file comentarios.
from google.colab import files
result_comentarios.to_csv('datos_limpios.csv',encoding='latin1', sep=';',index=False)
files.download('datos_limpios.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>